# Gunicorn project. ML

## Что умеет модель
Модель умеет предсказывать тип хотелки, введенной пользователем (юла, hh, ни то ни другое), далее по этому типу искать похожие объявления на соответствующей платформе.

Еще модель умеет предсказывать категорию для вакансии. Это не изюминка проекта, сделана для того чтобы а почему бы и нет.

In [1]:
%%time
import ml
model=ml.Model()

Wall time: 1min 14s


## Что за данные
Данные лежат в базе, в ноутбуке используется локальная копия (не уверен) для улучшения производительности.

In [2]:
%%time
import pandas as pd
data_hh=pd.read_csv(".data/hh_dataset.csv")
data_youla=pd.read_csv(".data/youla_dataset.csv")

Wall time: 1.06 s


In [27]:
data_hh.sample(1)

,id,title,category_id,category_name,employer,description,salary_from,salary_to,currency,metro_station
9405,31371325,Юрист,23,Юристы,НТО Центр промышленной безопасности,"ДЛЯ КОГО ЭТА ВАКАНСИЯ Мы ищем юриста, который ...",NaN,NaN,NaN,Волгоградский проспект


In [28]:
data_youla.sample(1)

,id,title,description,product_id,category_id,subcategory_id,properties,image_links
6523,6523,Зимняя куртка кожанная,Кожанка зимняя 54_56,5bb1d3fe0fff81421a6608d1,8.0,803,{'muzhskaya_odezhda_verhnyaya_tip': 'Кожаные к...,http://cache3.youla.io/files/images/360_360/5b...


## Поиск по юле
Для обучения использовался ансамбль BaggingClassifier на решающих деревьях. Векторизовались только названия объявлений, так как длинные описания усредняли вектор. Токенизация слов при помощи pymorphy2. Корпус слов для Word2Vec - Araneum, размерность вектора 300, алгоритм скипграм, обучен на 10млрд слов из [замечательного источника](https://www.google.ru/), взят [отсюда](https://rusvectores.org/static/models/rusvectores4/araneum/araneum_upos_skipgram_300_2_2018.vec.gz)

In [24]:
text="синяя куртка для девочки с капюшоном"
ids=model.knn_youla(text, 5)
data_youla.iloc[ids][['title','description']]

,title,description
23133,Куртка подростковая для девочки р.40-42,Куртка подростковая на девочку р.40-42. С капю...
5187,"Куртка зимняя ""MTFORCE""",Зимняя куртка для девочки.
6861,Куртка джинсовая,"Модная курточка , на девочку ."
8909,Куртка джинсовая,Продам джинсовую курточку на девочку.
35000,Куртка зимняя для девочки,Куртка зимняя для девочки. Подкладка- флис. Бе...


## Поиск по hh
Аналогично поиску по юле.

In [4]:
text="водитель автобуса"
ids=model.knn_hh(text, 5)
data_hh.iloc[ids][['title','description']]

,title,description
6329,Водитель категории Д на автобус,В транспортную компанию на постоянную работу т...
6192,Водитель-дальнобойщик,На постоянную работу требуется водитель-дально...
16020,Водитель такси Ford Focus,Крупной транспортной компании требуются води...
11685,Водитель-курьер с личным автомобилем,Обязанности: сбор и доставка почтовых отправл...
11706,Водитель Яндекс такси на зарплату,"В штат на зарплату. Водитель Такси, НОВЫЕ АВТО..."


## Определение типа хотелки
KNeighborsClassifier обучен на случайной 10 тыс. выборке с юлы и hh. Если максимальное значение не превышает критического, возвращается dummy-тип. Для ясности, печатаются вероятности принадлежности к hh и юле.

In [29]:
text="безумно дорогая и ненужная вещь"
print(model.wish_detector(text))

[0. 1.]
youla


In [30]:
text="Фронтенд программист без опыта"
print(model.wish_detector(text))

[1. 0.]
hh


In [32]:
text="фигня которую не определить"
print(model.wish_detector(text))

[0.4 0.6]
dummy


**ЗЫ** однако фигни на юле больше чем на hh

## Классификация вакансий
Снова бэггинг решающих деревьев, снова на рандомной 10тыс. выборке

In [7]:
text="Менеджер по продажам юридических услуг"
print(model.predict_text(text))

(17, 'Продажи')


In [10]:
data_hh.iloc[2656][['title','category_id','category_name']]

title            Менеджер по продажам юридических услуг
category_id                                           5
category_name                 Банки, инвестиции, лизинг
Name: 2656, dtype: object

по-моему, логичное предсказание

## Дамп данных
а именно:
* датасет юлы → анной + модель хотелки
* датасет hh → анной + модель хотелки
* категории hh → словарь + модель категории

In [6]:
%%time
ml.dump_all()

Wall time: 3min 46s


## Всё!
Оценки моделей лежат в [/model/ml2.ipynb](./ml2.ipynb)